1. Set up an environment for integration testing and write test cases that validate the interaction between different components of a system?

> Step 1 :
- Language: JavaScript / Node.js
- Framework: Express.js
- Testing Framework: Jest
- Integration Test Tool: Supertest (allows API testing without running a server manually)

> Step 2 : We will build a tiny system:
###Component A — User API
Handles creating and reading users.
###Component B — Database Layer
Stores users in an in-memory array (simulating a DB).

We will then write integration tests to check interaction between API layer and DB layer.

In [ ]:
npm init -y
npm install express jest supertest

Add this to package.json:

In [ ]:
{
  "scripts": {
    "test": "jest"
  }
}

db.js (Component B — Database)

In [ ]:
let users = [];

module.exports = {
  getUsers: () => users,

  addUser: (user) => {
    users.push(user);
    return user;
  },

  reset: () => { users = []; }
};

app.js (Component A — API)

In [ ]:
const express = require("express");
const db = require("./db");

const app = express();
app.use(express.json());

app.get("/users", (req, res) => {
  res.status(200).json(db.getUsers());
});

app.post("/users", (req, res) => {
  const user = db.addUser(req.body);
  res.status(201).json(user);
});

module.exports = app;

server.js

In [ ]:
const app = require("./app");
app.listen(3000, () => console.log("Server running on port 3000"));


3. Write Integration Test Cases

Integration tests ensure both API and Database work together correctly.

tests/integration.test.js

In [ ]:
const request = require("supertest");
const app = require("../app");
const db = require("../db");

beforeEach(() => {
  db.reset();   // ensure clean state for every test
});

describe("Integration Tests: API + DB", () => {

  test("POST /users → should create a new user", async () => {
    const response = await request(app)
      .post("/users")
      .send({ name: "Harsh", age: 22 });

    expect(response.status).toBe(201);
    expect(response.body.name).toBe("Harsh");
  });

  test("GET /users → should return all users", async () => {
    db.addUser({ name: "Aman", age: 25 });
    db.addUser({ name: "Rita", age: 28 });

    const response = await request(app).get("/users");

    expect(response.status).toBe(200);
    expect(response.body.length).toBe(2);
  });

});